In [14]:
from os import environ
from pathlib import Path
from json import loads, dumps
environ["OPENAI_API_KEY"] = Path("~/.openaiapikey").expanduser().read_text().strip()

In [15]:
from openai import OpenAI
from random import randint

openaiClient = OpenAI()
def gpt_3_5_turbo_completion(query):
    answer = openaiClient.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": query
            }
        ],
        seed = randint(0, 1000000)
    )
    return answer.choices[0].message.content

def gpt_4_turbo_completion(query):
    answer = openaiClient.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {
                "role": "system",
                "content": query
            }
        ],
        seed = randint(0, 1000000)
    )
    return answer.choices[0].message.content

def tryRecieveAnswer(query, completionFunction = gpt_4_turbo_completion, answerConversion = lambda x: True, maxTries = 10):
    tryNumber = 0
    while tryNumber < maxTries:
        answer = completionFunction(query)
        try:
            answer = answerConversion(answer)
            return (answer, True)
        except:
            pass
        tryNumber += 1
    print(f"Failed to recieve answer for query: {query}")
    return (None, False)

In [16]:
def giveQuestion(topic):
    query = f'Give me a question about the topic "{topic}". Return nothing but the question quoted in "".'
    def answerConversion(x):
        arr = x.split('"')
        assert len(arr) == 3
        assert arr[0] == "" and arr[2] == ""
        return arr[1]
    return tryRecieveAnswer(query, answerConversion = answerConversion)[0]

In [19]:
question = giveQuestion("Planet")
print(question)

What is the primary factor that determines a planet's ability to support life?


In [21]:
def giveAnswers(question, answerCount = 5):
    query = f'Give me {answerCount} different statements that are answering the question "{question}". Only the first statement has to be correct but all should sound serious. Return all statements in a list formatted as ["Statement 1", "Statement 2", ...]. Return nothing but the list.'  
    def answerConversion(x):
        x = x.strip()
        assert x[0] == "[" and x[-1] == "]"
        x = x[1:-1]
        x = x.split('"')
        assert all([x.strip() == ',' for x in x[2:-1:2]])
        assert x[0].strip() == x[-1].strip() == ""
        return x[1::2]
    return tryRecieveAnswer(query, answerConversion = answerConversion)[0]

In [22]:
answers = giveAnswers(question, 5)
print(dumps(answers, indent = 4))

[
    "The primary factor that determines a planet's ability to support life is the presence of liquid water, which requires temperatures and pressures that allow water to remain stable in its liquid form.",
    "The primary factor that determines a planet's ability to support life is the density of asteroids in its surrounding solar system, as frequent impacts can affect the evolution of life forms.",
    "The primary factor that determines a planet's ability to support life is the color of the planet\u2019s surface, as it influences the amount of solar energy absorbed and thus affects global temperature stability.",
    "The primary factor that determines a planet's ability to support life is the presence of magnetic fields, as they protect the planet from hostile cosmic rays and preserve its atmosphere.",
    "The primary factor that determines a planet's ability to support life is the composition of its core, which affects its gravitational pull and in turn the retention of an atmo

In [29]:
def getPartialStatements(statement):
    query = f'What are the partial statements that are contained in the statement "{statement}" and stand by them self? Return all partial statements in a list formatted as ["Partial Statement 1", "Partial Statement 2", ...]. If there are no partial statements the list should only contain the original statement. Return nothing but the list.'
    def answerConversion(x):
        x = x.strip()
        assert x[0] == "[" and x[-1] == "]"
        x = x[1:-1]
        x = x.split('"')
        assert all([x.strip() == ',' for x in x[2:-1:2]])
        assert x[0].strip() == x[-1].strip() == ""
        return x[1::2]
    return tryRecieveAnswer(query, answerConversion = answerConversion)[0]

In [30]:
partialStatements = getPartialStatements(answers[2])
print(dumps(partialStatements, indent = 4))

[
    "The primary factor that determines a planet's ability to support life is the color of the planet\u2019s surface.",
    "The color of the planet\u2019s surface influences the amount of solar, energy absorbed.",
    "The amount of solar energy absorbed affects global temperature stability."
]


In [31]:
subPartialStatements = getPartialStatements(partialStatements[0])
print(dumps(subPartialStatements, indent = 4))

[
    "The primary factor that determines a planet's ability to support life is the color of the planet\u2019s surface."
]


In [40]:
def findCorrespondingQuestion(statement):
    query = f'Formulate a question that is answered by the statement "{statement}". Return nothing but the question quoted in "".'
    def answerConversion(x):
        arr = x.split('"')
        assert len(arr) == 3
        assert arr[0] == "" and arr[2] == ""
        return arr[1]
    return tryRecieveAnswer(query, answerConversion = answerConversion)[0]

In [43]:
question = findCorrespondingQuestion(partialStatements[2])
print(question)

What impacts global temperature stability in relation to solar energy?


True
